$ max{\,8.1x_1\, + \,10.8x_2} $
$s.t.$
$0.8$

In [6]:
from pyomo.environ import *

class OptiModel:
    
    def __init__(self):
        self.m = ConcreteModel()
        self.m.dual = Suffix(direction=Suffix.IMPORT)
        self.m.x1 = Var(domain=NonNegativeReals)
        self.m.x2 = Var(domain=NonNegativeReals)
        self.m.obj = Objective(
            expr=- 8.1*self.m.x1 - 10.8*self.m.x2,
            sense=minimize
        )
        self.m.c1 = Constraint(rule=self.c1_rule)
        self.m.c2 = Constraint(rule=self.c2_rule)
        self.m.c3 = Constraint(rule=self.c3_rule)
        
        solverpath = 'C:\\w64\\glpsol'
        self.solver = SolverFactory('glpk', executable=solverpath)
#         self.solver = SolverManagerFactory('neos')
        
#         self.m.constraints = ConstraintList()
#         self.m.constraints.add(
#             0.8*self.m.x1 + 0.44*self.m.x2  <= 24000.0 + 6000.2
#         )
        
#         self.m.constraints.add(
#             0.05*self.m.x1 + 0.1*self.m.x2 <= 2000.0 + 409.84
#         )
        
#         self.m.constraints.add(
#             0.1*self.m.x1 + 0.36*self.m.x2 <= 6000.0
#         )
        
#         self.m.obj = Objective(
#             expr=- 8.1*self.m.x1 - 10.8*self.m.x2,
#             sense=minimize
#         )

    def c1_rule(self, m):
        return 0.8*m.x1 + 0.44*m.x2  <= 24000.0 + 6000.2
    
    def c2_rule(self, m):
        return 0.05*m.x1 + 0.1*m.x2 <= 2000.0 + 409.84
    
    def c3_rule(self, m):
        return 0.1*m.x1 + 0.36*m.x2 <= 6000.0
        

        
    def solve(self):
        self.solver.solve(self.m, tee=True)
#         self.solver.solve(self.m, opt='ipopt', tee=True, keepfiles=True, logfile='mylog.log')
    
    def write(self):
        print(self.m.x1.value, self.m.x2.value)

In [7]:
mymodel = OptiModel()
mymodel.solve()
mymodel.write()

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\user1\AppData\Local\Temp\tmpnq12l2dd.glpk.raw --wglp C:\Users\user1\AppData\Local\Temp\tmpemckgh8c.glpk.glp
 --cpxlp C:\Users\user1\AppData\Local\Temp\tmpqxmqzntv.pyomo.lp
Reading problem data from 'C:\Users\user1\AppData\Local\Temp\tmpqxmqzntv.pyomo.lp'...
4 rows, 3 columns, 7 non-zeros
31 lines were read
Writing problem data to 'C:\Users\user1\AppData\Local\Temp\tmpemckgh8c.glpk.glp'...
23 lines were written
GLPK Simplex Optimizer, v4.65
4 rows, 3 columns, 7 non-zeros
Preprocessing...
3 rows, 2 columns, 6 non-zeros
Scaling...
 A: min|aij| =  5.000e-02  max|aij| =  8.000e-01  ratio =  1.600e+01
GM: min|aij| =  6.252e-01  max|aij| =  1.600e+00  ratio =  2.558e+00
EQ: min|aij| =  3.909e-01  max|aij| =  1.000e+00  ratio =  2.558e+00
Constructing initial basis...
Size of triangular part is 3
*     0: obj =   0.000000000e+00 inf =   0.000e+00 (2)
*     2: obj =  -3.505586524e+05 inf =   0.000e+0

In [8]:
# fill M, m by n matrix. 
# m is no. of variables + no. of active constraints
# n is also no. of variables + no. of active constraints
# so over all it must be a square matrix.

i = 1
# print(mymodel.m.constraints[i]._body)
mymodel.m.dual.display()#
# print(list(mymodel.m.component_data_objects(Var)))
# expr.current.decompose_term((mymodel.m.constraints[i]._body))
# list(mymodel.m.dual


dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key : Value
     c1 :  -4.6551724137931
     c2 : -87.5172413793103
     c3 :               0.0


In [191]:
import numpy as np

# get no of variables
var_list = list(mymodel.m.component_data_objects(Var))
no_of_var = len(var_list)
print('no_of_var=' + str(no_of_var)) 

# get duals as list
no_of_constr = len(mymodel.m.constraints)
print('no_of_constr=' + str(no_of_constr))

dual_list = np.zeros(shape=[no_of_constr])
# note duals are defined in opposite sign as linear program codes tend to use Ax >= b where as we use g(x) <= 0
for i in range(no_of_constr):
    dual_list[i] = - mymodel.m.dual[mymodel.m.constraints[i+1]]

print(dual_list)
active_constr_list = [mymodel.m.constraints[i+1] for i in range(0, len(dual_list)) if dual_list[i] > 0]
no_of_active_constr = len(active_constr_list)

active_dual_list = dual_list[dual_list != 0.0]


# get shape of M
M_len = no_of_var + no_of_active_constr

# Define M as 
# [ 0            0            c_x1_c1  c_x1_c2
#   .            .            c_x2_c1  c_x2_c2
#   c_x1_c1(y1)  c_x2_c1(y1)  0        0
#   c_x1_c2(y2)  c_x2_c2(y2)  0        0        ]
M = np.zeros(shape=[M_len, M_len])

# convert outer tuple to list, then inner tuples
for i in range(no_of_active_constr):
    coeff_var = expr.current.decompose_term(active_constr_list[i]._body)[1]
    coeff_var = [list(j) for j in coeff_var]

    coeff_var = np.array(coeff_var)
    coeff_var = coeff_var[:, 0].astype(float)

    M[0:no_of_var, no_of_var + i] = coeff_var.T
    M[no_of_var + i, 0:no_of_var] = coeff_var * active_dual_list[i]

# Define N as matrix 
N = np.zeros(shape=[M_len, 2])
N[no_of_var + 0, 0] = -active_dual_list[0]
N[no_of_var + 1, 1] = -active_dual_list[1]

y = np.linalg.solve(M, N)


no_of_var=2
no_of_constr=3
[ 4.65517241 87.51724138 -0.        ]


array([[ -1.72413793,   7.5862069 ],
       [  0.86206897, -13.79310345],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ]])

In [200]:
theta0 = np.array([6000.2, 409.84])
constant = np.dot(-y, -theta0.T) + np.r_[[i.value for i in var_list], active_dual_list]
slope = -y


[33442.9379310345, 7376.93103448276]

!!!!! Note that currently when inserting values into M in the first v rows, we still need to figure out a way to insert zero for variables that don't exist (have a coeff of 0) in a constraint. <br/>
!!!!! Note that we still need a way to identify which theta belongs to which constraint.
